In [2]:
import pandas as pd

In [54]:
vehicle_count = pd.read_excel(r"C:\Users\hai\Desktop\SPECTRATHON\data\10962-ev-registration-counts-by-state_8-2-23.xlsx")

port_count = pd.read_csv(r"C:\Users\hai\Desktop\SPECTRATHON\data\alt_fuel_stations.csv")
stations = list(port_count.groupby(["State"]).count()["Groups With Access Code"])
states = sorted(list(set(list(port_count['State']))))
cars = (list(vehicle_count["Registration Count"]))
for i in range(len(cars)):
    if type(cars[i])==str:
        cars[i]=int(cars[i].replace(',',''))


In [85]:
stations

[66,
 448,
 330,
 1255,
 16652,
 2270,
 974,
 391,
 207,
 3486,
 2039,
 385,
 385,
 217,
 1360,
 604,
 559,
 322,
 281,
 3136,
 1765,
 503,
 1503,
 808,
 1263,
 170,
 131,
 1637,
 98,
 270,
 271,
 1352,
 311,
 606,
 4032,
 1663,
 364,
 1291,
 1802,
 16,
 363,
 589,
 98,
 952,
 3339,
 913,
 1615,
 397,
 2311,
 604,
 162]

In [21]:
import csv

In [28]:
area = [52419,
663267,
113998,
53179,
163695,
104093,
5544,
2489,
68,
65755,
59425,
10931,
83570,
57914,
36418,
56271,
82276,
40410,
51840,
35385,
12407,
10555,
96716,
86938,
48431,
69704,
147042,
77353,
110560,
9350,
8722,
121589,
54556,
53818,
70700,
44825,
69899,
98380,
46056,
3515,
1545,
32020,
77353,
42144,
268580,
3794083,
84898,
9615,
42774,
71300,
24230,
65498,
97813]

In [29]:
pop = [4779736,
710231,
6392017,
2915918,
37253956,
5029196,
3574097,
897934,
601723,
18801310,
9687653,
1360301,
1567582,
12830632,
6483802,
3046355,
2853118,
4339367,
4533372,
1328361,
5773552,
6547629,
9883640,
5303925,
2967297,
5988144,
989415,
1826341,
2700551,
1316470,
8791894,
2059179,
19378102,
9535483,
672591,
11536504,
3751351,
3831074,
12702379,
3725789,
1052567,
4625364,
814180,
6346165,
25145561,
308745538,
2763885,
625741,
8001024,
6724540,
1852994,
5686986,
563626]

In [79]:
states=states[:51]
area=area[:51]
pop=pop[:51]
cars=cars[:51]
stations=stations[:51]
thresh=100
x_train = []
y_train = []
x_test = []
y_test = []
with open("predict.csv","w") as y:
    w = csv.writer(y)
    w.writerow(("Location","Area","Population","Reg_cars","Stations"))
    with open("network.csv", 'w') as f:
        wr = csv.writer(f)
        wr.writerow(("Location","Area","Population","Reg_cars","Stations"))
        for i in range(len(states)):
            if area[i]*cars[i]/pop[i]>thresh:
                x_train.append((area[i],pop[i],cars[i]))
                y_train.append(stations[i])
                wr.writerow((states[i],area[i],pop[i],cars[i],stations[i]))
            else:
                x_test.append((area[i],pop[i],cars[i]))
                y_test.append(stations[i])
                w.writerow((states[i],area[i],pop[i],cars[i]))


    

In [77]:
from sklearn.linear_model import LogisticRegression

In [80]:
logreg= LogisticRegression()
logreg.fit(x_train,y_train)

c:\Users\hai\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [84]:
y_pred=logreg.predict(x_test)
print(y_pred)

[3339 3339 3486 3486 3486  311 1503 3339 3486 3486 3339 2039 3486   98
  311 3486  311 3339  311 3339]


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
import csv

In [7]:
import pandas as pd
import plotly.express as px

# Load your data here. Replace 'data.csv' with your actual file path
# Ensure your data has columns for 'fips' (state identifiers) and a value column (e.g., 'unemployment_rate')
data = pd.read_csv('network.csv')

# Get the GeoJSON data for US states
states_geojson = "https://github.com/PublicaMundi/MappingAPI/blob/master/data/geojson/us-states.json"

# Create the choropleth map
fig = px.choropleth(
    data,
    geojson=states_geojson,
    locations='Location',  # Replace with your state identifier column name
    color='Stations',  # Replace with your data value column name
    range_color=[min(data['Stations']), max(data['Stations'])],
    scope="usa",
    title='USA Choropleth Map'
)

# Customize the map (optional)
fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))  # Remove padding

# Display the map
fig.show()